In [1]:
import os 
os.environ['CUDA_VISIBLE_DEVICES']="-1"
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = "false"


In [2]:
import numpy as np
import math
from functools import partial
import pickle

from jax import tree_map

In [3]:
#44, 256, 256, 1

In [4]:
def SIREN(x, params):
    weights1, weights2, weights3, mu_x, var_x, mu_t, var_t = params
    x = (x - mu_x) / var_x

    def forward(x, weights):

        weights = weights['params']

        for key in list(weights.keys())[:-1]:
            x = x@weights[key]['kernel'] + weights[key]['bias']
            x = np.sin(x)

        key = list(weights.keys())[-1]
        x = x@weights[key]['kernel'] + weights[key]['bias']
        return x
    
    outputs = []

    for wbs in [weights1, weights2, weights3]:
        outputs.append(forward(x, wbs))

    outputs = np.concatenate(outputs, -1)
    return outputs * var_t + mu_t


In [5]:
# models = np.load('../model.pkl', allow_pickle=True)
models = pickle.load(open('../jax_pinn.pkl', 'rb'))
models = tree_map(lambda x : np.array(x), models)
np.save('../jax_pinns/jax_pinn_model', models, allow_pickle=True)

/scratch/wangh19/anaconda3/lib/python3.8/site-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [6]:
forward_fn = partial(SIREN, params = models)
x = np.ones((1, 13,))
y = forward_fn(x)

y

array([[25.84501881,  7.87353072, 13.04773458]])